<a href="https://colab.research.google.com/github/MileGjorgiev/VNP_Proekt_211093/blob/main/VNP_Standarden_Proekt_211093_T6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from bs4.element import Tag

In [2]:
boxSite = 'https://box.co.uk/laptops?brand_filter=616451-616449&product_list_limit=36'
boxSiteResponse = requests.get(boxSite)
boxSiteRaw_html = boxSiteResponse.text
boxSiteSoup = BeautifulSoup(boxSiteRaw_html, "html.parser")
boxSiteProducts = boxSiteSoup.select('app-product-list-view-card')
boxSiteProductsParsed_products = []

In [3]:
def extract_product_characteristics_as_dict_boxSite(product):
  alt_text = product.select_one('img[alt]')['alt']
  match = re.match(r'^\/.\/.\/([a-zA-Z0-9]+)_logo', alt_text)
  if match:
    brand = match.group(1).capitalize()

  name = product.select_one('a.line-clamp-3').text.strip()
  specs_ul = product.select_one('.specs')
  processor = None
  ram = None
  storage = None
  gpu = None

  if specs_ul:
      specs_list = specs_ul.find_all('li')
      for spec in specs_list:
          text = spec.get_text(strip=True)

          if 'processor' in text.lower():
              processor = text

          if re.search(r'\b\d+\s?gb\s?ram\b', text.lower()):
              match = re.search(r'\d+\s?GB\s?RAM', text, re.IGNORECASE)
              if match:
                  ram = match.group()

          if re.search(r'\b\d+\s?(tb|gb)\s?ssd\b', text.lower()):
              match = re.search(r'\d+\s?(TB|GB)\s?SSD', text, re.IGNORECASE)
              if match:
                  storage = match.group()

          if 'nvidia' in text.lower() or 'graphics' in text.lower():
              gpu = text

  price_tag = product.select_one('span.text-3xl.text-heading_primary.font-semibold')
  price = price_tag.text.strip() if price_tag else None


  return {
      'brand' : brand,
      'name': name if name else None,
      'processor': processor,
      'ram': ram,
      'storage': storage,
      'gpu': gpu,
      'price' : price
  }

In [4]:
for product in boxSiteProducts:
  result = extract_product_characteristics_as_dict_boxSite(product)
  boxSiteProductsParsed_products.append(result)
boxSiteProductsParsed_products

[{'brand': 'Lenovo',
  'name': 'Lenovo Legion 5 Intel Core i7-14650HX 16GB RAM 1TB SSD RTX 4070 16" Gaming Laptop',
  'processor': 'Intel Core i7 -14650HX 16-Core Processor',
  'ram': '16 GB RAM',
  'storage': '1 TB SSD',
  'gpu': 'NVIDIA GeForce RTX 4070  Graphics',
  'price': '£1299.99'},
 {'brand': 'Lenovo',
  'name': 'Lenovo ThinkBook 16 AMD Ryzen 5 7535HS 8GB RAM 256GB SSD 16" Laptop',
  'processor': 'AMD Ryzen 5 7535HS Hexa-Core Processor',
  'ram': '8 GB RAM',
  'storage': '256 GB SSD',
  'gpu': 'AMD Radeon 660M  Graphics',
  'price': '£449.99'},
 {'brand': 'Lenovo',
  'name': 'Lenovo V14 Intel Core i5-13420H 24GB RAM 512GB SSD 14" Laptop',
  'processor': 'Intel Core i5 -13420H Octa-Core Processor',
  'ram': '24 GB RAM',
  'storage': '512 GB SSD',
  'gpu': 'Intel UHD  Graphics',
  'price': '£539.99'},
 {'brand': 'Lenovo',
  'name': 'Lenovo V15 Ryzen AMD Ryzen 5 7520U 16GB RAM 256GB SSD 15.6" Laptop',
  'processor': 'AMD Ryzen 5 7520U Quad-Core Processor',
  'ram': '16 GB RAM',
 

In [5]:
computerlounge = 'https://computerlounge.co.nz/collections/business-laptops'
computerloungeResponse = requests.get(computerlounge)
computerlounge_html = computerloungeResponse.text
computerloungeSoup = BeautifulSoup(computerlounge_html, "html.parser")
computerloungeProducts = computerloungeSoup.select('.card--product-contained')
computerloungeProductsParsed_products = []

In [6]:
def extract_product_characteristics_as_dict_computerlounge(product):
  name = product.select_one('.card__title a').text.strip()
  brand = name.split()[0].capitalize()
  specs_block = product.select_one('.product-info__block')
  processor = None
  ram = None
  storage = None
  gpu = None

  if specs_block:
        spec_labels = specs_block.select('span.product-label.product-label--pc')
        for label in spec_labels:
            text = label.get_text(strip=True)

            if re.search(r'(ultra|ryzen|core|X)', text.lower()):
                processor = text

            if re.search(r'\b\d+\s?gb\s?ram\b', text.lower()):
                match = re.search(r'\d+\s?GB\s?RAM', text, re.IGNORECASE)
                if match:
                    ram = match.group()

            if re.search(r'\b\d+\s?(tb|gb)\s?ssd\b', text.lower()):
                match = re.search(r'\d+\s?(TB|GB)\s?SSD', text, re.IGNORECASE)
                if match:
                    storage = match.group()

            if re.search(r'(graphics|nvidia|radeon|intel arc)', text.lower()):
                gpu = text

  price_tag = product.select_one('.price__current')
  price = price_tag.text.strip() if price_tag else None


  return {
      'brand' : brand,
      'name': name if name else None,
      'processor': processor,
      'ram': ram,
      'storage': storage,
      'gpu': gpu,
      'price' : price
  }

In [7]:
computerloungeBaseUrl = 'https://computerlounge.co.nz/collections/business-laptops?page='
for i in range(1, 4):
  url = computerloungeBaseUrl + str(i)
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  products = soup.select('.card--product-contained')
  for product in products:
    result = extract_product_characteristics_as_dict_computerlounge(product)
    computerloungeProductsParsed_products.append(result)
computerloungeProductsParsed_products

[{'brand': 'Asus',
  'name': 'ASUS Expertbook B1402CBA-EB0089X 14" FHD i5-1235U16GB 512GB W11 PRO Business Laptop',
  'processor': 'Core i5-1235U',
  'ram': '16GB RAM',
  'storage': '512GB SSD',
  'gpu': 'Intel Graphics',
  'price': '$1,149.00'},
 {'brand': 'Hp',
  'name': 'HP EliteBook 665 G11 16" Touchscreen Business Laptop',
  'processor': 'Ryzen 5 7535U',
  'ram': '16GB RAM',
  'storage': '512GB SSD',
  'gpu': 'AMD Graphics',
  'price': '$1,877.00'},
 {'brand': 'Hp',
  'name': 'HP EliteBook 660 G11 16" Business Laptop',
  'processor': 'Ultra 7 155U',
  'ram': '16GB RAM',
  'storage': '512GB SSD',
  'gpu': 'Intel Graphics',
  'price': '$2,215.00'},
 {'brand': 'Msi',
  'name': "MSI Prestige 16 AI Evo B1MG-050NZ 16'' QHD+ Laptop",
  'processor': 'Ultra 7 155H',
  'ram': '32GB RAM',
  'storage': '1TB SSD',
  'gpu': 'Intel Graphics',
  'price': '$2,999.00'},
 {'brand': 'Hp',
  'name': 'HP ProBook 460 G11 16" FHD+ Touchscreen Business Laptop',
  'processor': 'Ultra 7 155H',
  'ram': '16G

In [8]:
laptopsdirect = 'https://www.laptopsdirect.co.uk/ct/laptops-and-netbooks/laptops/everyday?pageNumber=3'
laptopsdirectResponse = requests.get(laptopsdirect)
laptopsdirect_html = laptopsdirectResponse.text
laptopsdirectSoup = BeautifulSoup(laptopsdirect_html, "html.parser")
laptopsdirectProducts = laptopsdirectSoup.select('.OfferBox')
laptopsdirectProductsParsed_products = []

In [9]:
def extract_product_characteristics_as_dict_laptopsdirect(product):
  name = product.select_one('.offerboxtitle').text.strip()
  brand = name.split()[0].capitalize()
  if brand == 'Refurbished':
    brand = name.split()[1].capitalize()
  specs_ul = product.select_one('.productInfo ul')
  processor = None
  ram = None
  storage = None
  gpu = None

  if specs_ul:
      specs_list = specs_ul.find_all('li')
      for spec in specs_list:
          title = spec.select_one('.proddesctitle')
          value = spec.select_one('.proddescvalue')
          value_text = value.get_text(strip=True) if value else None
          title_text = title.get_text(strip=True).lower() if title else ''

          if 'processor' in title_text:
              processor = value_text
          elif 'ram' in title_text:
              ram = value_text
          elif ('ssd' in title_text or
          'storage' in title_text or
          'hard drive' in title_text or
          (value_text and re.search(r'\b\d+\s?(gb|tb)\b', value_text.lower()))):
            # storage = value_text
            title_text = title_text.strip() if title_text else ''
            if title_text == 'hard drive':
              title_text = 'HDD'
            value_text = value_text.strip() if value_text else ''
            storage = f"{value_text} {title_text}".strip().upper()
          elif 'graphics' in title_text or 'gpu' in title_text:
              gpu = value_text

  price_tag = product.select_one('.offerprice')
  price = price_tag.text.strip() if price_tag else None


  return {
      'brand' : brand,
      'name': name if name else None,
      'processor': processor,
      'ram': ram,
      'storage': storage,
      'gpu': gpu,
      'price' : price
  }

In [ ]:
laptopsdirectBaseUrl = 'https://www.laptopsdirect.co.uk/nav/pt/laptops/brd/asus/mc/48361/lbu/everyday?pageNumber='
for i in range(1, 4):
  url = laptopsdirectBaseUrl + str(i)
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  products = soup.select('.OfferBox')
  for product in products:
    result = extract_product_characteristics_as_dict_laptopsdirect(product)
    laptopsdirectProductsParsed_products.append(result)
laptopsdirectProductsParsed_products

In [11]:
laptopsdirectDf = pd.DataFrame(laptopsdirectProductsParsed_products)
computerloungeDf = pd.DataFrame(computerloungeProductsParsed_products)
boxSiteDf = pd.DataFrame(boxSiteProductsParsed_products)

In [12]:
laptopsdirectDf = laptopsdirectDf[laptopsdirectDf['processor'].notna()]
def fill_gpu_laptopsdirect(row):
    gpu = row['gpu']
    proc = row['processor'].lower()

    proc = proc.replace('n series', 'nseries').strip()

    if pd.isna(gpu) or gpu.strip().lower() == 'unknown integrated gpu' or gpu.strip() == '':
        if any(x in proc for x in ['intel', 'core', 'celeron', 'pentium', 'nseries']):
            if any(x in proc for x in ['i7', 'i5', 'i3']):
                return 'Intel Iris Xe Graphics'
            elif any(x in proc for x in ['celeron', 'pentium', 'nseries']):
                return 'Intel UHD Graphics'
            else:
                return 'Intel UHD Graphics'
        elif any(x in proc for x in ['ryzen', 'athlon']):
            return 'Integrated AMD Radeon Graphics'
        elif 'mediatek' in proc:
            return 'Mediatek Integrated Graphics'
        else:
            return 'Unknown Integrated GPU'
    else:
        return gpu

laptopsdirectDf['gpu'] = laptopsdirectDf.apply(fill_gpu_laptopsdirect, axis=1)

In [13]:
computerloungeDf = computerloungeDf[computerloungeDf['processor'].notna()]
computerloungeDf.at[10, 'gpu'] = 'Intel Graphics'

In [14]:
computerloungeDf

,brand,name,processor,ram,storage,gpu,price
0,Asus,"ASUS Expertbook B1402CBA-EB0089X 14"" FHD i5-12...",Core i5-1235U,16GB RAM,512GB SSD,Intel Graphics,"$1,149.00"
1,Hp,"HP EliteBook 665 G11 16"" Touchscreen Business ...",Ryzen 5 7535U,16GB RAM,512GB SSD,AMD Graphics,"$1,877.00"
2,Hp,"HP EliteBook 660 G11 16"" Business Laptop",Ultra 7 155U,16GB RAM,512GB SSD,Intel Graphics,"$2,215.00"
3,Msi,MSI Prestige 16 AI Evo B1MG-050NZ 16'' QHD+ La...,Ultra 7 155H,32GB RAM,1TB SSD,Intel Graphics,"$2,999.00"
4,Hp,"HP ProBook 460 G11 16"" FHD+ Touchscreen Busine...",Ultra 7 155H,16GB RAM,512GB SSD,Intel Graphics,"$2,064.48"
5,Hp,"HP ProBook 460 G11 16"" FHD+ Business Laptop",Ultra 7 155U,16GB RAM,256GB SSD,Intel Graphics,"$1,896.23"
6,Hp,"HP ProBook x360 435 G10 13.3"" FHD Touchscreen ...",Ryzen 5 7530U,16GB RAM,256GB SSD,AMD Graphics,"$1,588.84"
9,Msi,MSI Commercial 14 H A13MG vPro-027NZ i5-13500H...,Core i5-13500H,16GB RAM,512GB SSD,Intel Graphics,"$2,099.00"
10,Msi,MSI Summit E16 AI Studio A1VFTG-051NZ 16'' QHD...,Ultra 7 155H,16GB RAM,1TB SSD,Intel Graphics,"$4,299.00"
11,Msi,"MSI Modern 14 F13MG-014NZ 14"" FHD Laptop",Core i7-1355U,16GB RAM,512GB SSD,Intel Graphics,"$1,399.00"


In [15]:
boxSiteDf

,brand,name,processor,ram,storage,gpu,price
0,Lenovo,Lenovo Legion 5 Intel Core i7-14650HX 16GB RAM...,Intel Core i7 -14650HX 16-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 4070 Graphics,£1299.99
1,Lenovo,Lenovo ThinkBook 16 AMD Ryzen 5 7535HS 8GB RAM...,AMD Ryzen 5 7535HS Hexa-Core Processor,8 GB RAM,256 GB SSD,AMD Radeon 660M Graphics,£449.99
2,Lenovo,Lenovo V14 Intel Core i5-13420H 24GB RAM 512GB...,Intel Core i5 -13420H Octa-Core Processor,24 GB RAM,512 GB SSD,Intel UHD Graphics,£539.99
3,Lenovo,Lenovo V15 Ryzen AMD Ryzen 5 7520U 16GB RAM 25...,AMD Ryzen 5 7520U Quad-Core Processor,16 GB RAM,256 GB SSD,AMD Radeon 610M Graphics,£399.99
4,Lenovo,Lenovo Legion 5 Intel Core i7-14650HX 16GB RAM...,Intel Core i7 -14650HX 16-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 4060 Graphics,£1099.99
5,Lenovo,Lenovo LOQ 15 Intel Core i5-12450H 16GB RAM 51...,Intel Core i5 -12450H Octa-Core Processor,16 GB RAM,512 GB SSD,NVIDIA GeForce RTX 4060 Graphics,£799.99
6,Acer,Acer Predator Helios Neo 18 AI Intel Core Ultr...,Intel Core Ultra 9 275HX 24-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 5070 Ti Graphics,£2099.99
7,Lenovo,Lenovo ThinkPad E16 Ryzen AMD Ryzen 5 7535HS 1...,AMD Ryzen 5 7535HS Hexa-Core Processor,16 GB RAM,256 GB SSD,AMD Radeon 660M Graphics,£619.99
8,Lenovo,Lenovo ThinkPad E14 AMD Ryzen 5 7535HS 16GB RA...,AMD Ryzen 5 7535HS Hexa-Core Processor,16 GB RAM,256 GB SSD,AMD Radeon 660M Graphics,£599.99
9,Acer,Acer Predator Helios Neo 16 Ai Intel Core Ultr...,Intel Core Ultra 9 275HX 24-Core Processor,32 GB RAM,1 TB SSD,NVIDIA GeForce RTX 5070 Ti Graphics,£1999.99


In [16]:
laptopsdirectDf

,brand,name,processor,ram,storage,gpu,price
0,Asus,Refurbished ASUS Vivobook 15 X1505ZA Core i5-1...,Core i5 12500H,16GB,1TB SSD,Intel Iris Xe Graphics,£399.97
1,Asus,ASUS VivoBook 16 M1605YA AMD Ryzen 8GB RAM 512...,Ryzen 5 7530u,8GB,512GB SSD,Integrated AMD Radeon Graphics,£369.97
2,Asus,ASUS Vivobook Go 15 Intel N 4GB RAM 128GB SSD ...,N Series N200,4GB,128GB HDD,Intel UHD Graphics,£209.97
3,Asus,ASUS VivoBook X1504VA-NJ513W Intel Core i7 16G...,Core i7 1355U,16GB,1TB SSD,Intel Iris Xe Graphics,£583.97
5,Asus,ASUS Vivobook Go 15 AMD Ryzen 3 8GB RAM 128GB ...,Ryzen 3 7320U,8GB,128GB SSD,AMD Radeon 610M,£363.97
6,Asus,ASUS 15 Intel Celeron N N4500 4GB RAM 64GB eMM...,CELERON N N4500,4GB,64GB HDD,Intel UHD Graphics,£319.97
7,Asus,ASUS E510MAEJ040WS Intel Celeron 4GB RAM 15.6 ...,Celeron N4020,4GB,64GB HDD,Intel UHD Graphics,£239.97
8,Asus,ASUS CX34 Intel Core i3 8GB RAM 256GB SSD 14 I...,Core i3 1215U,8GB,256GB SSD,Intel Iris Xe Graphics,£649.97
9,Asus,ASUS Vivobook Go 14 Intel Celeron 4GB RAM 128G...,Celeron N4500,4GB,128GB HDD,Intel UHD Graphics,£249.97
11,Asus,ASUS ExpertBook Intel Core i5 16GB RAM 512GB S...,Core i5 1335U,16GB,512GB SSD,Intel Iris Xe Graphics,£669.97


In [17]:
def poundToMKD(DF):
  DF['price'] = DF['price'].replace('[£,]', '', regex=True).astype(float)
  exchange_rate = 71.5
  DF['price'] = (DF['price'] * exchange_rate).round().astype(int)
  DF['price'] = DF['price'].astype(str) + ' ден'


In [18]:
poundToMKD(laptopsdirectDf)
laptopsdirectDf

,brand,name,processor,ram,storage,gpu,price
0,Asus,Refurbished ASUS Vivobook 15 X1505ZA Core i5-1...,Core i5 12500H,16GB,1TB SSD,Intel Iris Xe Graphics,28598 ден
1,Asus,ASUS VivoBook 16 M1605YA AMD Ryzen 8GB RAM 512...,Ryzen 5 7530u,8GB,512GB SSD,Integrated AMD Radeon Graphics,26453 ден
2,Asus,ASUS Vivobook Go 15 Intel N 4GB RAM 128GB SSD ...,N Series N200,4GB,128GB HDD,Intel UHD Graphics,15013 ден
3,Asus,ASUS VivoBook X1504VA-NJ513W Intel Core i7 16G...,Core i7 1355U,16GB,1TB SSD,Intel Iris Xe Graphics,41754 ден
5,Asus,ASUS Vivobook Go 15 AMD Ryzen 3 8GB RAM 128GB ...,Ryzen 3 7320U,8GB,128GB SSD,AMD Radeon 610M,26024 ден
6,Asus,ASUS 15 Intel Celeron N N4500 4GB RAM 64GB eMM...,CELERON N N4500,4GB,64GB HDD,Intel UHD Graphics,22878 ден
7,Asus,ASUS E510MAEJ040WS Intel Celeron 4GB RAM 15.6 ...,Celeron N4020,4GB,64GB HDD,Intel UHD Graphics,17158 ден
8,Asus,ASUS CX34 Intel Core i3 8GB RAM 256GB SSD 14 I...,Core i3 1215U,8GB,256GB SSD,Intel Iris Xe Graphics,46473 ден
9,Asus,ASUS Vivobook Go 14 Intel Celeron 4GB RAM 128G...,Celeron N4500,4GB,128GB HDD,Intel UHD Graphics,17873 ден
11,Asus,ASUS ExpertBook Intel Core i5 16GB RAM 512GB S...,Core i5 1335U,16GB,512GB SSD,Intel Iris Xe Graphics,47903 ден


In [19]:
poundToMKD(boxSiteDf)
boxSiteDf

,brand,name,processor,ram,storage,gpu,price
0,Lenovo,Lenovo Legion 5 Intel Core i7-14650HX 16GB RAM...,Intel Core i7 -14650HX 16-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 4070 Graphics,92949 ден
1,Lenovo,Lenovo ThinkBook 16 AMD Ryzen 5 7535HS 8GB RAM...,AMD Ryzen 5 7535HS Hexa-Core Processor,8 GB RAM,256 GB SSD,AMD Radeon 660M Graphics,32174 ден
2,Lenovo,Lenovo V14 Intel Core i5-13420H 24GB RAM 512GB...,Intel Core i5 -13420H Octa-Core Processor,24 GB RAM,512 GB SSD,Intel UHD Graphics,38609 ден
3,Lenovo,Lenovo V15 Ryzen AMD Ryzen 5 7520U 16GB RAM 25...,AMD Ryzen 5 7520U Quad-Core Processor,16 GB RAM,256 GB SSD,AMD Radeon 610M Graphics,28599 ден
4,Lenovo,Lenovo Legion 5 Intel Core i7-14650HX 16GB RAM...,Intel Core i7 -14650HX 16-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 4060 Graphics,78649 ден
5,Lenovo,Lenovo LOQ 15 Intel Core i5-12450H 16GB RAM 51...,Intel Core i5 -12450H Octa-Core Processor,16 GB RAM,512 GB SSD,NVIDIA GeForce RTX 4060 Graphics,57199 ден
6,Acer,Acer Predator Helios Neo 18 AI Intel Core Ultr...,Intel Core Ultra 9 275HX 24-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 5070 Ti Graphics,150149 ден
7,Lenovo,Lenovo ThinkPad E16 Ryzen AMD Ryzen 5 7535HS 1...,AMD Ryzen 5 7535HS Hexa-Core Processor,16 GB RAM,256 GB SSD,AMD Radeon 660M Graphics,44329 ден
8,Lenovo,Lenovo ThinkPad E14 AMD Ryzen 5 7535HS 16GB RA...,AMD Ryzen 5 7535HS Hexa-Core Processor,16 GB RAM,256 GB SSD,AMD Radeon 660M Graphics,42899 ден
9,Acer,Acer Predator Helios Neo 16 Ai Intel Core Ultr...,Intel Core Ultra 9 275HX 24-Core Processor,32 GB RAM,1 TB SSD,NVIDIA GeForce RTX 5070 Ti Graphics,142999 ден


In [20]:
def dolarToMKD(DF):
  DF['price'] = DF['price'].replace('[$,]', '', regex=True).astype(float)
  exchange_rate = 53.3
  DF['price'] = (DF['price'] * exchange_rate).round().astype(int)
  DF['price'] = DF['price'].astype(str) + ' ден'

In [21]:
dolarToMKD(computerloungeDf)
computerloungeDf

,brand,name,processor,ram,storage,gpu,price
0,Asus,"ASUS Expertbook B1402CBA-EB0089X 14"" FHD i5-12...",Core i5-1235U,16GB RAM,512GB SSD,Intel Graphics,61242 ден
1,Hp,"HP EliteBook 665 G11 16"" Touchscreen Business ...",Ryzen 5 7535U,16GB RAM,512GB SSD,AMD Graphics,100044 ден
2,Hp,"HP EliteBook 660 G11 16"" Business Laptop",Ultra 7 155U,16GB RAM,512GB SSD,Intel Graphics,118060 ден
3,Msi,MSI Prestige 16 AI Evo B1MG-050NZ 16'' QHD+ La...,Ultra 7 155H,32GB RAM,1TB SSD,Intel Graphics,159847 ден
4,Hp,"HP ProBook 460 G11 16"" FHD+ Touchscreen Busine...",Ultra 7 155H,16GB RAM,512GB SSD,Intel Graphics,110037 ден
5,Hp,"HP ProBook 460 G11 16"" FHD+ Business Laptop",Ultra 7 155U,16GB RAM,256GB SSD,Intel Graphics,101069 ден
6,Hp,"HP ProBook x360 435 G10 13.3"" FHD Touchscreen ...",Ryzen 5 7530U,16GB RAM,256GB SSD,AMD Graphics,84685 ден
9,Msi,MSI Commercial 14 H A13MG vPro-027NZ i5-13500H...,Core i5-13500H,16GB RAM,512GB SSD,Intel Graphics,111877 ден
10,Msi,MSI Summit E16 AI Studio A1VFTG-051NZ 16'' QHD...,Ultra 7 155H,16GB RAM,1TB SSD,Intel Graphics,229137 ден
11,Msi,"MSI Modern 14 F13MG-014NZ 14"" FHD Laptop",Core i7-1355U,16GB RAM,512GB SSD,Intel Graphics,74567 ден


In [22]:
allProductsDf = pd.concat([boxSiteDf, computerloungeDf, laptopsdirectDf], ignore_index=True)
allProductsDf.columns = ['Brand', 'Name', 'Processor', 'RAM', 'Storage', 'GPU', 'Price']
allProductsDf

,Brand,Name,Processor,RAM,Storage,GPU,Price
0,Lenovo,Lenovo Legion 5 Intel Core i7-14650HX 16GB RAM...,Intel Core i7 -14650HX 16-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 4070 Graphics,92949 ден
1,Lenovo,Lenovo ThinkBook 16 AMD Ryzen 5 7535HS 8GB RAM...,AMD Ryzen 5 7535HS Hexa-Core Processor,8 GB RAM,256 GB SSD,AMD Radeon 660M Graphics,32174 ден
2,Lenovo,Lenovo V14 Intel Core i5-13420H 24GB RAM 512GB...,Intel Core i5 -13420H Octa-Core Processor,24 GB RAM,512 GB SSD,Intel UHD Graphics,38609 ден
3,Lenovo,Lenovo V15 Ryzen AMD Ryzen 5 7520U 16GB RAM 25...,AMD Ryzen 5 7520U Quad-Core Processor,16 GB RAM,256 GB SSD,AMD Radeon 610M Graphics,28599 ден
4,Lenovo,Lenovo Legion 5 Intel Core i7-14650HX 16GB RAM...,Intel Core i7 -14650HX 16-Core Processor,16 GB RAM,1 TB SSD,NVIDIA GeForce RTX 4060 Graphics,78649 ден
...,...,...,...,...,...,...,...
119,Grade,GRADE A1 - ASUS Zenbook S 13 Intel Core Ultra ...,Core Ultra 7 155U,16GB,1TB SSD,Intel UHD Graphics,95093 ден
120,Asus,ASUS Vivobook Go 14 Intel Celeron 4GB RAM 128G...,Celeron N4500,4GB,128GB HDD,Intel UHD Graphics,12153 ден
121,Asus,ASUS CX34 Intel Core i5 8GB RAM 256 SSD 14 Inc...,Core i5 1235U,8GB,256GB SSD,Intel Iris Xe Graphics,47545 ден
122,Asus,ASUS Vivobook Go 15 Intel Core i3 8GB RAM 128G...,Core i3 N305,8GB,128GB SSD,Intel Iris Xe Graphics,20733 ден


In [23]:
allProductsDf.to_csv('AllLaptops.csv', index=False)